In [2]:
# Opinion Mining / Sentiment Analysis
# technique: bag-of-word
# 1. tokenize every documents: raw documents are converted into individual words/roots set.
# 2. create feature matrix depending on frequency of tokens(words) occurency in particular documents

# load movie comments dataset
# import pyprind
import pandas as pd
# import os

# basepath = 'aclImdb'

# labels = {'pos':1, 'neg':0}
# pbar = pyprind.ProgBar(50000)
# df = pd.DataFrame()
# for s in ('test', 'train'):
#     for l in ('pos', 'neg'):
#         path = os.path.join(basepath, s, l)
#         for file in sorted(os.listdir(path)):
#             with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
#                 txt = infile.read()
#             df = df.append([[txt, labels[l]]], ignore_index=True)
#             pbar.update()
# df.columns = ['review', 'sentiment']

In [3]:
# store comments as csv file
import numpy as np
# np.random.seed(0)
# df = df.reindex(np.random.permutation(df.index))
# df.to_csv('movie_data.csv', index=False, encoding='utf-8')
df = pd.read_csv('movie_data.csv', encoding='utf-8')
print(df.head(3))
print(df.shape)

                                              review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0
(50000, 2)


In [4]:
# clean up dataset

# 1. remove html tokens and non-word chars
# 2. store emoticons
# 3. lower the word 
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

print(preprocessor(df.loc[0, 'review'][-50:]))
df['review'] = df['review'].apply(preprocessor)

is seven title brazil not available


In [5]:
# bag-of-word

# 1. tokenize 
# ------------
# create vocabulary of documents('review' in this case)

text = df.loc[0, 'review'][:100]
print(text)

# a. simply split every work according to 'space'
def tokenizer(text):
    return text.split()
print('Simply Split: ', tokenizer(text))

# b. word stemming: convert word into root
# porter stemmer
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
print('Porter Stemmer: ', tokenizer_porter(text))

# snowball stemmer
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer(language='english')
def tokenizer_snowball(text):
    return [snowball.stem(word) for word in text.split()]
print('SnowBall Stemmer: ', tokenizer_snowball(text))

# lancaster stemmer
from nltk.stem.lancaster import LancasterStemmer
lancaster = LancasterStemmer()
def tokenizer_lancaster(text):
    return [lancaster.stem(word) for word in text.split()]
print('lancaster Stemmer: ', tokenizer_lancaster(text))

# stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwic
Simply Split:  ['in', '1974', 'the', 'teenager', 'martha', 'moxley', 'maggie', 'grace', 'moves', 'to', 'the', 'high', 'class', 'area', 'of', 'belle', 'haven', 'greenwic']
Porter Stemmer:  ['in', '1974', 'the', 'teenag', 'martha', 'moxley', 'maggi', 'grace', 'move', 'to', 'the', 'high', 'class', 'area', 'of', 'bell', 'haven', 'greenwic']
SnowBall Stemmer:  ['in', '1974', 'the', 'teenag', 'martha', 'moxley', 'maggi', 'grace', 'move', 'to', 'the', 'high', 'class', 'area', 'of', 'bell', 'haven', 'greenwic']
lancaster Stemmer:  ['in', '1974', 'the', 'teen', 'marth', 'moxley', 'maggy', 'grac', 'mov', 'to', 'the', 'high', 'class', 'are', 'of', 'bel', 'hav', 'greenw']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\12923\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# 2. tf, tf-idf
# -------------------------------------------------------
# transform words/vocabularies into numerical features

# a. tf
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['Ths sun is shining', 
                 'The weather is sweet', 
                 'The sun is shining, the weather is sweet', 
                 'and one and one is two'])
tf = count.fit_transform(docs)
print(count.vocabulary_)
print(tf.toarray())

# b. tf-idf, weighted tf
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
np.set_printoptions(precision=3)
print(tfidf.fit_transform(tf).toarray())

{'ths': 7, 'sun': 4, 'is': 1, 'shining': 3, 'the': 6, 'weather': 9, 'sweet': 5, 'and': 0, 'one': 2, 'two': 8}
[[0 1 0 1 1 0 0 1 0 0]
 [0 1 0 0 0 1 1 0 0 1]
 [0 2 0 1 1 1 2 0 0 1]
 [2 1 2 0 0 0 0 0 1 0]]
[[0.    0.329 0.    0.497 0.497 0.    0.    0.631 0.    0.   ]
 [0.    0.357 0.    0.    0.    0.539 0.539 0.    0.    0.539]
 [0.    0.424 0.    0.32  0.32  0.32  0.64  0.    0.    0.32 ]
 [0.657 0.171 0.657 0.    0.    0.    0.    0.    0.328 0.   ]]


In [7]:
# training model with bag-of-word

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values
print(X_train)


from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, 
                        lowercase=False, 
                        preprocessor=None)
para_grid = [{
              'vect__ngram_range':[(1, 1)], 
              'vect__stop_words':[stop, None], 
              'vect__tokenizer':[tokenizer, tokenizer_porter], 
              'clf__penalty':['l1', 'l2'], 
              'clf__C':[1.0, 10.0, 100.0]
              }, 
             {
              'vect__ngram_range':[(1, 1)], 
              'vect__stop_words':[stop, None], 
              'vect__tokenizer':[tokenizer, tokenizer_porter], 
              'vect__use_idf':[False], 
              'vect__norm':[None], 
              'clf__penalty':['l1', 'l2'], 
              'clf__C':[1.0, 10.0, 100.0]
             }
            ]
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, solver='liblinear'))])
gs = GridSearchCV(lr_tfidf, para_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=-1)
gs.fit(X_train, y_train)  # briefly 24m32.3s

['in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of halloween she was murdered in the backyard of her house and her murder remained unsolved twenty two years later the writer mark fuhrman christopher meloni who is a former la detective that has fallen in disgrace for perjury in o j simpson trial and moved to idaho decides to investigate the case with his partner stephen weeks andrew mitchell with the purpose of writing a book the locals squirm and do not welcome them but with the support of the retired detective steve carroll robert forster that was in charge of the investigation in the 70 s they discover the criminal and a net of power and money to cover the murder murder in greenwich is a good tv movie with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a kennedy the powerful and rich family used their influence to cover the mu

KeyboardInterrupt: 

In [ ]:
print('Best parameter Set: %s ' % gs.best_params_)
print('CV Accuracy: %.3f' % gs.best_score_)
clf = gs.best_estimator_
print('Test Accuracy: %.3f ' % clf.score(X_test, y_test))

Best parameter Set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001F7F445DBD0>} 
CV Accuracy: 0.897
Test Accuracy: 0.899 
